model

In [1]:
!pip install transformers==3.0.2

In [3]:
import torch
from transformers import get_linear_schedule_with_warmup, BertForSequenceClassification, BertConfig
from transformers import get_cosine_schedule_with_warmup

C:\Users\user\anaconda3\envs\ml\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [4]:
from torch import nn
from transformers import BertModel, BertConfig

class BertClassifier(nn.Module):

    # BertClassifier는 초대용량 AI 모델인 BertModel을 파인튜닝한 모델이다.
    # 구체적으로, BertModel 위에 linear한 층 하나를 추가해 놓은 모델이다.
    # Bert -> Linear -> Softmax의 순서로 연산이 이루어진다.
    # 우리가 Loss를 구하는데 사용되는 CrossEntropyLoss의 경우, softmax를 자동으로 포함하여 연산해준다.
    # 고로, BertClassifier 클래스 내의 forward 함수에, softmax는 포함시키지 않는다.
    # 단, 학습의 용도가 아니라 테스트나 실제 시연을 할 때는 forward 결과물에 softmax를 적용시켜줘야 한다.

    def __init__(self, num_labels=7):

        super(BertClassifier, self).__init__()

        model_config = BertConfig.from_pretrained('monologg/kobert',attention_probs_dropout_prob = 0.1,
                                                  hidden_dropout_prob = 0.1,classifier_dropout = 0.1)
        
        # model_config와 from_pretrained 함수를 활용해서 bert 모델을 선언해주자.
        # self.bert = ~~ 꼴로 작성해주자.
        ## 여기에 코드 작성
        self.bert = BertModel(model_config)
        # nn 라이브러리를 활용해서 linear 모델을 선언해주자.
        # self.linear = ~~ 꼴로 작성해주자.
        # bert의 경우, 768차원 벡터를 산출해준다.
        # linear는 768차원의 값을 입력받아 7차원의 값을 출력해주어야 한다.
        ## 여기에 코드 작성
        self.linear1=nn.Linear(768,128)
        self.relu=nn.GELU()
        self.drop = nn.Dropout(p=0.1)
        self.linear2 = nn.Linear(128, num_labels)

    def forward(self, input_ids, attention_mask):
        # 위에서 생성한 self.bert, self.linear 두 개의 layer를 활용하면 된다.
        # bert -> linear를 적용해준 다음, 출력값을 리턴해주자.
        # Hint: 만약 __init__에서 self.bert라는 bert layer를 생성하였고 이를 forward 함수에서 활용하려면, 생성한 bert layer의 forward 함수를 활용하면 될 것이다.
        #       bert layer의 forward 함수는 input_ids와 attention_mask를 변수로 입력받아야 한다. 이에 따라, self.bert(input_ids, attention_mask)와 같은 방식으로 코드를 작성하면 될 것이다.
        ## 여기에 코드 작성
        _,x = self.bert(input_ids, attention_mask)

        x = self.linear1(x)
        x=self.relu(x)
        x = self.drop(x)
        x=self.linear2(x)
        
        return x

In [5]:
# 분류를 위한 BERT 모델 생성
# BertModel을 초기화하는 역할
def BertModelInitialization():
    PATH = "./model.pt"
    
    # BertModel은 다양한 작업을 진행할 수 있도록 해주는 여러 인터페이스들을 제공한다.
    # 그 중, 본 중간 미션의 task에 가장 적합한 인터페이스를 찾아보자.
    # Hint 1 : HuggingFace 홈페이지에 Bert를 검색해서 찾아보자.
    # URL : https://huggingface.co/docs/transformers/main/en/index
    # Hint 2 : 본 중간 미션이, 영화리뷰를 긍정과 부정의 두 가지 감정으로 분류해내는 작업임을 고려해보자.
    
    #model = BertClassifier()
    model = BertForSequenceClassification.from_pretrained('monologg/kobert', num_labels=7)
    # 생성한 모델을 특정 PATH에 저장
    torch.save(model.state_dict(), PATH)

In [6]:
def get_model(device, cuda_available):
    PATH = "model.pt"
    
    #model = BertClassifier()
    model = BertForSequenceClassification.from_pretrained('monologg/kobert', num_labels=7)

    if cuda_available:
        # PATH에 저장된 모델 불러옴
        model.load_state_dict(torch.load(PATH), strict=False)

        # 불러온 모델을 device에 등록
        model = model.to(device)
    else:
        # PATH에 저장된 모델을 불러오기 및 불러온 모델을 device에 등록
        model.load_state_dict(torch.load(PATH, map_location=device), strict=False)
    
    return model

**신경망 성능 향상을 위한 다양한 툴**



> 최종미션을 위한 이론문서를 확인해보자.


- 여러분은 옵티마이저, 에포크 수, 스케줄러 종류를 본 task에 가장 적합한 것으로 선택하여 바꿀 수 있다.
- 정답은 없다. 최적의 모델을 선택하여 신경망의 성능을 높여보자!




In [7]:
def get_model_with_params(num_data, device, cuda_available):
    model = get_model(device, cuda_available)

    # 옵티마이저 설정
    optimizer = torch.optim.Adam(model.parameters(),
                      lr = 5e-5, # 학습률
                      eps = 1e-8, # 0으로 나누는 것을 방지하기 위한 epsilon 값,
                    )
    # 전체 데이터가 총 몇 번 학습되는지
    epochs = 5
    warmup_ratio = 0.1


    # 총 훈련 스텝
    total_steps = num_data * epochs
    warmup_steps = int(total_steps* warmup_ratio)
    # 학습이 이루어짐에 따라 learning_rate을 감소시키기 위한 스케줄러
    scheduler = get_cosine_schedule_with_warmup(optimizer, 
                                                num_warmup_steps = warmup_steps,
                                                num_training_steps = total_steps)
  
    # nn 라이브러리에서 crossentropy 형식으로 loss를 계산하는 모델을 찾아서 손실함수로 설정해주자.
    # criterion = ~~ 꼴로 작성해주자.
    ## 여기에 코드 작성
    # 손실 함수 정의
    criterion = nn.CrossEntropyLoss()

    
    return model, optimizer, scheduler, epochs, criterion


In [8]:
!pip install import_ipynb

In [9]:
#%cd /content/gdrive/MyDrive/Colab Notebooks/OUTTA
#!ls

In [10]:

import import_ipynb

from nlp_Preproc_final import preproc
import random
import numpy as np
import pandas as pd
import torch
from tqdm.notebook import tqdm
import time
from tokenization_kobert import KoBertTokenizer

importing Jupyter notebook from nlp_Preproc_final.ipynb
  Using cached numpy-1.16.6-cp38-cp38-win_amd64.whl
  Attempting uninstall: numpy
    Found existing installation: numpy 1.23.4
    Uninstalling numpy-1.23.4:
      Successfully uninstalled numpy-1.23.4


ERROR: Could not install packages due to an OSError: [WinError 5] 액세스가 거부되었습니다: 'C:\\Users\\user\\anaconda3\\envs\\ml\\Lib\\site-packages\\numpy\\~.2bs\\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll'
Consider using the `--user` option or check the permissions.



  Using cached numpy-1.23.4-cp38-cp38-win_amd64.whl (14.7 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.16.6
    Uninstalling numpy-1.16.6:
      Successfully uninstalled numpy-1.16.6


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.9.1 requires tensorboard<2.10,>=2.9, but you have tensorboard 2.10.0 which is incompatible.
tensorboard 2.10.0 requires requests<3,>=2.21.0, but you have requests 2.18.4 which is incompatible.
scipy 1.7.3 requires numpy<1.23.0,>=1.16.5, but you have numpy 1.23.4 which is incompatible.
numba 0.55.1 requires numpy<1.22,>=1.18, but you have numpy 1.23.4 which is incompatible.
mxnet 1.7.0.post2 requires numpy<1.17.0,>=1.8.2, but you have numpy 1.23.4 which is incompatible.


2.9.1


In [11]:
def main():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    BertModelInitialization()
    model = get_model(device, torch.cuda.is_available())

    print(get_model_with_params(29991, device, cuda_available = True))

In [12]:
if __name__ == '__main__':
      main()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


(BertClassifier(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(8002, 768, padding_idx=1)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=Tru